# Data Sets

The data is contained in three files:

* portfolio.json - containing offer ids and meta data about each offer (duration, type, etc.)
* profile.json - demographic data for each customer
* transcript.json - records for transactions, offers received, offers viewed, and offers completed

Here is the schema and explanation of each variable in the files:

**portfolio.json**
* id (string) - offer id
* offer_type (string) - type of offer ie BOGO, discount, informational
* difficulty (int) - minimum required spend to complete an offer
* reward (int) - reward given for completing an offer
* duration (int) - time for offer to be open, in days
* channels (list of strings)

**profile.json**
* age (int) - age of the customer 
* became_member_on (int) - date when customer created an app account
* gender (str) - gender of the customer (note some entries contain 'O' for other rather than M or F)
* id (str) - customer id
* income (float) - customer's income

**transcript.json**
* event (str) - record description (ie transaction, offer received, offer viewed, etc.)
* person (str) - customer id
* time (int) - time in hours since start of test. The data begins at time t=0
* value - (dict of strings) - either an offer id or transaction amount depending on the record


In [2]:
import os
import pandas as pd

%matplotlib inline

# read in the json files
# profile = pd.read_json('data/profile.json', orient='records', lines=True)
transcript = pd.read_json('data/transcript.json', orient='records', lines=True)

## Portfolio: Offers

In [55]:
portfolio = pd.read_json('data/portfolio.json', orient='records', lines=True)
portfolio.rename(columns={'id': 'offer_id'}, inplace=True)
portfolio['duration'] = portfolio['duration'] * 24
portfolio

# id (string) - offer id
# offer_type (string) - type of offer ie BOGO, discount, informational
# difficulty (int) - minimum required spend to complete an offer
# reward (int) - reward given for completing an offer
# duration (int) - time for offer to be open, in days
# channels (list of strings)

,reward,channels,difficulty,duration,offer_type,offer_id
0,10,"[email, mobile, social]",10,168,bogo,ae264e3637204a6fb9bb56bc8210ddfd
1,10,"[web, email, mobile, social]",10,120,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0
2,0,"[web, email, mobile]",0,96,informational,3f207df678b143eea3cee63160fa8bed
3,5,"[web, email, mobile]",5,168,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9
4,5,"[web, email]",20,240,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7
5,3,"[web, email, mobile, social]",7,168,discount,2298d6c36e964ae4a3e7e9706d1fb8c2
6,2,"[web, email, mobile, social]",10,240,discount,fafdcd668e3743c1bb461111dcafc2a4
7,0,"[email, mobile, social]",0,72,informational,5a8bc65990b245e5a138643cd4eb9837
8,5,"[web, email, mobile, social]",5,120,bogo,f19421c1d4aa40978ebb69ca19b0e20d
9,2,"[web, email, mobile]",10,168,discount,2906b810c7d4411798c6938adc9daaa5


In [4]:
transcript.head()

# event (str) - record description (ie transaction, offer received, offer viewed, etc.)
# person (str) - customer id
# time (int) - time in hours since start of test. The data begins at time t=0
# value - (dict of strings) - either an offer id or transaction amount depending on the record

,person,event,value,time
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,{'offer id': '9b98b8c7a33c4b65b9aebfe6a799e6d9'},0
1,a03223e636434f42ac4c3df47e8bac43,offer received,{'offer id': '0b1e1539f2cc45b7b9fa7c272da2e1d7'},0
2,e2127556f4f64592b11af22de27a7932,offer received,{'offer id': '2906b810c7d4411798c6938adc9daaa5'},0
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,{'offer id': 'fafdcd668e3743c1bb461111dcafc2a4'},0
4,68617ca6246f4fbc85e91a2a49552598,offer received,{'offer id': '4d5c57ea9a6940dd891ad53e9dbe8da0'},0


In [5]:
transcript.event.value_counts()

transaction        138953
offer received      76277
offer viewed        57725
offer completed     33579
Name: event, dtype: int64

## Offers Funnel Data

In [12]:
def create_transcript_copy(df) -> pd.DataFrame:
    # make copy of transcript
    df = transcript.copy()
    # flatten JSON and extract value 
    df['value_flat'] = df['value'].apply(lambda x: list(x.items())[0][1])
    # rename and drop columns
    df = df.rename(columns={'person': 'customer_id'}).drop(columns=['value'])
    return df

df_transcript = create_transcript_copy(transcript)
df_transcript.head()

,customer_id,event,time,value_flat
0,78afa995795e4d85b5d9ceeca43f5fef,offer received,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,offer received,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,offer received,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,offer received,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,offer received,0,4d5c57ea9a6940dd891ad53e9dbe8da0


In [13]:
def create_df_base_received(df_transcript) -> pd.DataFrame:
    rename_columns = {'event': 'offer_received', 'value_flat': 'offer_id', 'time': 'offer_received_time'}
    df_received = df_transcript[df_transcript['event']=='offer received'].rename(columns=rename_columns)
    df_received['offer_received'] = df_received['offer_received'].apply(lambda x: 1 if not pd.isnull(x) else 0)
    return df_received

df_received = create_df_base_received(df_transcript)
df_received.head()

,customer_id,offer_received,offer_received_time,offer_id
0,78afa995795e4d85b5d9ceeca43f5fef,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
1,a03223e636434f42ac4c3df47e8bac43,1,0,0b1e1539f2cc45b7b9fa7c272da2e1d7
2,e2127556f4f64592b11af22de27a7932,1,0,2906b810c7d4411798c6938adc9daaa5
3,8ec6ce2a7e7949b1bf142def7d0e0586,1,0,fafdcd668e3743c1bb461111dcafc2a4
4,68617ca6246f4fbc85e91a2a49552598,1,0,4d5c57ea9a6940dd891ad53e9dbe8da0


In [15]:
df_received.offer_received_time.describe()

count    76277.000000
mean       332.579519
std        196.489548
min          0.000000
25%        168.000000
50%        408.000000
75%        504.000000
max        576.000000
Name: offer_received_time, dtype: float64

In [20]:
# number of customers receiving offers at time 0, 168, ...
df_received.groupby('offer_received_time')['customer_id'].count().reset_index()

,offer_received_time,customer_id
0,0,12650
1,168,12669
2,336,12711
3,408,12778
4,504,12704
5,576,12765


In [22]:
# number of customers receiving each offer at time = 0
df_received[df_received['offer_received_time']==0].groupby('offer_id')['customer_id'].nunique().reset_index()

,offer_id,customer_id
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,1297
1,2298d6c36e964ae4a3e7e9706d1fb8c2,1221
2,2906b810c7d4411798c6938adc9daaa5,1257
3,3f207df678b143eea3cee63160fa8bed,1273
4,4d5c57ea9a6940dd891ad53e9dbe8da0,1218
5,5a8bc65990b245e5a138643cd4eb9837,1266
6,9b98b8c7a33c4b65b9aebfe6a799e6d9,1245
7,ae264e3637204a6fb9bb56bc8210ddfd,1292
8,f19421c1d4aa40978ebb69ca19b0e20d,1263
9,fafdcd668e3743c1bb461111dcafc2a4,1318


In [27]:
# count number of offers received by offer_id
offer_received = df_received.groupby('offer_id')['offer_received'].sum().reset_index()
offer_received

,offer_id,offer_received
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,7668
1,2298d6c36e964ae4a3e7e9706d1fb8c2,7646
2,2906b810c7d4411798c6938adc9daaa5,7632
3,3f207df678b143eea3cee63160fa8bed,7617
4,4d5c57ea9a6940dd891ad53e9dbe8da0,7593
5,5a8bc65990b245e5a138643cd4eb9837,7618
6,9b98b8c7a33c4b65b9aebfe6a799e6d9,7677
7,ae264e3637204a6fb9bb56bc8210ddfd,7658
8,f19421c1d4aa40978ebb69ca19b0e20d,7571
9,fafdcd668e3743c1bb461111dcafc2a4,7597


In [28]:
# create base df offer viewed; df_viewed = create_df_base_viewed(df_transcript)
def create_df_base_viewed(df_transcript) -> pd.DataFrame:
    rename_columns = {'event': 'offer_viewed', 'value_flat': 'offer_id', 'time': 'offer_viewed_time'}
    df_viewed = df_transcript[df_transcript['event']=='offer viewed'].rename(columns=rename_columns)
    df_viewed['offer_viewed'] = df_viewed['offer_viewed'].apply(lambda x: 1 if not pd.isnull(x) else 0)
    return df_viewed

df_viewed = create_df_base_viewed(df_transcript)
offer_viewed = df_viewed.groupby('offer_id')['offer_viewed'].sum().reset_index()
offer_viewed

,offer_id,offer_viewed
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,2663
1,2298d6c36e964ae4a3e7e9706d1fb8c2,7337
2,2906b810c7d4411798c6938adc9daaa5,4118
3,3f207df678b143eea3cee63160fa8bed,4144
4,4d5c57ea9a6940dd891ad53e9dbe8da0,7298
5,5a8bc65990b245e5a138643cd4eb9837,6687
6,9b98b8c7a33c4b65b9aebfe6a799e6d9,4171
7,ae264e3637204a6fb9bb56bc8210ddfd,6716
8,f19421c1d4aa40978ebb69ca19b0e20d,7264
9,fafdcd668e3743c1bb461111dcafc2a4,7327


In [29]:
# create base df offer completed; df_completed = create_df_base_completed(df_transcript)
def create_df_base_completed(df_transcript) -> pd.DataFrame:
    rename_columns={'time':'offer_completed_time', 'value_flat':'offer_id'}
    df_completed = df_transcript[df_transcript['event']=='offer completed'].rename(columns=rename_columns)
    # create new column as second column location with all values 1
    df_completed.insert(loc=1, column='offer_completed', value=1)
    df_completed.drop(columns=['event'], inplace=True)
    return df_completed

df_completed = create_df_base_completed(df_transcript)
offer_completed = df_completed.groupby('offer_id')['offer_completed'].sum().reset_index()
offer_completed

,offer_id,offer_completed
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,3420
1,2298d6c36e964ae4a3e7e9706d1fb8c2,5156
2,2906b810c7d4411798c6938adc9daaa5,4017
3,4d5c57ea9a6940dd891ad53e9dbe8da0,3331
4,9b98b8c7a33c4b65b9aebfe6a799e6d9,4354
5,ae264e3637204a6fb9bb56bc8210ddfd,3688
6,f19421c1d4aa40978ebb69ca19b0e20d,4296
7,fafdcd668e3743c1bb461111dcafc2a4,5317


## Transactions with offers completed only

In [31]:
# df_transactions = create_df_transactions(df_transcript)
import uuid

def create_df_transactions(df_transcript) -> pd.DataFrame:
    rename_columns={'time':'transaction_time', 'value_flat':'transaction_amount'}

    # create dataframe with only transaction events
    df_transactions = df_transcript[df_transcript['event']=='transaction'].rename(columns=rename_columns)

    # convert transaction_amount into float
    df_transactions['transaction_amount'] = df_transactions['transaction_amount'].apply(lambda x: float(x))

    # create transaction_id's and drop event column 
    transaction_ids = []
    n = len(df_transactions)
    for _ in range(n):
        _id = uuid.uuid4().hex
        transaction_ids.append(_id)

    # add transaction_id as first column in df_transactions
    df_transactions.insert(loc=0, column='transaction_id', value=transaction_ids)
    df_transactions.drop(columns=['event'], inplace=True)
    
    return df_transactions

df_transactions = create_df_transactions(df_transcript)

df_engagement_v1 = pd.merge(df_transactions, 
                            df_completed, 
                            how='inner', 
                            left_on=['customer_id','transaction_time'],
                            right_on=['customer_id','offer_completed_time'],
                            suffixes=['','_drop'])

df_engagement_v1['offer_completed'] = df_engagement_v1['offer_completed'].fillna(0).astype(int)
df_engagement_v1['offer_completed_time'] = df_engagement_v1['offer_completed_time'].fillna(-1).astype(int)
df_engagement_v1['offer_id'] = df_engagement_v1['offer_id'].fillna('no-offer')
df_engagement_v1.head()

,transaction_id,customer_id,transaction_time,transaction_amount,offer_completed,offer_completed_time,offer_id
0,1ac07baf2a9943bebe6bd8013b7834a8,9fa9ae8f57894cc9a3b8a9bbe0fc1b2f,0,34.56,1,0,2906b810c7d4411798c6938adc9daaa5
1,74518a8c59b743a9a51a6b10d8ccae7b,fe97aa22dd3e48c8b143116a8403dd52,0,18.97,1,0,fafdcd668e3743c1bb461111dcafc2a4
2,9ca27f36dbc44ae2b3fc47606783f842,629fc02d56414d91bca360decdfa9288,0,33.90,1,0,9b98b8c7a33c4b65b9aebfe6a799e6d9
3,c0c6c21e3644498db61251dac86fd302,676506bad68e4161b9bbaffeb039626b,0,18.01,1,0,ae264e3637204a6fb9bb56bc8210ddfd
4,1d8d7ca4bbda4f699a9ff08f8e2715c4,8f7dd3b2afe14c078eb4f6e6fe4ba97d,0,19.11,1,0,4d5c57ea9a6940dd891ad53e9dbe8da0


In [36]:
df_engagement_v1.transaction_amount.sum()

687925.4400000001

In [45]:
offer_transaction_cnt = df_engagement_v1.groupby('offer_id')['transaction_id'].nunique().reset_index()
offer_transaction_cnt.rename(columns={'transaction_id': 'transaction_cnt'}, inplace=True)
offer_transaction_cnt

,offer_id,transaction_cnt
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,3340
1,2298d6c36e964ae4a3e7e9706d1fb8c2,5112
2,2906b810c7d4411798c6938adc9daaa5,3964
3,4d5c57ea9a6940dd891ad53e9dbe8da0,3301
4,9b98b8c7a33c4b65b9aebfe6a799e6d9,4303
5,ae264e3637204a6fb9bb56bc8210ddfd,3635
6,f19421c1d4aa40978ebb69ca19b0e20d,4262
7,fafdcd668e3743c1bb461111dcafc2a4,5265


In [46]:
offer_transaction_amount = df_engagement_v1.groupby('offer_id')['transaction_amount'].sum().reset_index()
offer_transaction_amount

,offer_id,transaction_amount
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,87898.52
1,2298d6c36e964ae4a3e7e9706d1fb8c2,90552.42
2,2906b810c7d4411798c6938adc9daaa5,82688.83
3,4d5c57ea9a6940dd891ad53e9dbe8da0,79283.59
4,9b98b8c7a33c4b65b9aebfe6a799e6d9,77911.04
5,ae264e3637204a6fb9bb56bc8210ddfd,88106.63
6,f19421c1d4aa40978ebb69ca19b0e20d,83592.17
7,fafdcd668e3743c1bb461111dcafc2a4,97892.24


In [47]:
offer_customer_cnt = df_engagement_v1.groupby('offer_id')['customer_id'].nunique().reset_index()
offer_customer_cnt.rename(columns={'customer_id': 'customer_cnt'}, inplace=True)
offer_customer_cnt

,offer_id,customer_cnt
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,2978
1,2298d6c36e964ae4a3e7e9706d1fb8c2,4421
2,2906b810c7d4411798c6938adc9daaa5,3480
3,4d5c57ea9a6940dd891ad53e9dbe8da0,2885
4,9b98b8c7a33c4b65b9aebfe6a799e6d9,3784
5,ae264e3637204a6fb9bb56bc8210ddfd,3177
6,f19421c1d4aa40978ebb69ca19b0e20d,3741
7,fafdcd668e3743c1bb461111dcafc2a4,4530


## Create Offer Funnel

### Offers Funnel Columns:
* offer_received_count
* offer_viewed_count
* offer_completed_count
* transaction_count
* customer_count
* total transaction amount
--------------------
* viewed_rate
* completion_rate
* avg_transactions_per_customer
* avg_spend_per_customer
--------------------
* total reward
* total difficulty 
* reward-difficulty ratio >> higher the ratio more reward money

In [49]:
offer_funnel = offer_received
merge_list = [offer_viewed, offer_completed, offer_transaction_cnt, offer_transaction_amount, offer_customer_cnt]
for data in merge_list:
    offer_funnel = offer_funnel.merge(data, how='left', on='offer_id').fillna(0)
    
offer_funnel    

,offer_id,offer_received,offer_viewed,offer_completed,transaction_cnt,transaction_amount,customer_cnt
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,7668,2663,3420.0,3340.0,87898.52,2978.0
1,2298d6c36e964ae4a3e7e9706d1fb8c2,7646,7337,5156.0,5112.0,90552.42,4421.0
2,2906b810c7d4411798c6938adc9daaa5,7632,4118,4017.0,3964.0,82688.83,3480.0
3,3f207df678b143eea3cee63160fa8bed,7617,4144,0.0,0.0,0.00,0.0
4,4d5c57ea9a6940dd891ad53e9dbe8da0,7593,7298,3331.0,3301.0,79283.59,2885.0
5,5a8bc65990b245e5a138643cd4eb9837,7618,6687,0.0,0.0,0.00,0.0
6,9b98b8c7a33c4b65b9aebfe6a799e6d9,7677,4171,4354.0,4303.0,77911.04,3784.0
7,ae264e3637204a6fb9bb56bc8210ddfd,7658,6716,3688.0,3635.0,88106.63,3177.0
8,f19421c1d4aa40978ebb69ca19b0e20d,7571,7264,4296.0,4262.0,83592.17,3741.0
9,fafdcd668e3743c1bb461111dcafc2a4,7597,7327,5317.0,5265.0,97892.24,4530.0


In [53]:
offer_funnel['viewed_rate'] = 1.0 * offer_funnel['offer_viewed'] / offer_funnel['offer_received']
offer_funnel['completion_rate'] = 1.0 * offer_funnel['offer_completed'] / offer_funnel['offer_received']
offer_funnel['avg_transactions_per_customer'] = 1.0 * offer_funnel['transaction_cnt'] / offer_funnel['customer_cnt']
offer_funnel['avg_spend_per_customer'] = 1.0 * offer_funnel['transaction_amount'] / offer_funnel['customer_cnt']
offer_funnel = offer_funnel.fillna(0)
offer_funnel


,offer_id,offer_received,offer_viewed,offer_completed,transaction_cnt,transaction_amount,customer_cnt,viewed_rate,completion_rate,avg_transactions_per_customer,avg_spend_per_customer
0,0b1e1539f2cc45b7b9fa7c272da2e1d7,7668,2663,3420.0,3340.0,87898.52,2978.0,0.347287,0.446009,1.121558,29.515957
1,2298d6c36e964ae4a3e7e9706d1fb8c2,7646,7337,5156.0,5112.0,90552.42,4421.0,0.959587,0.674340,1.156299,20.482339
2,2906b810c7d4411798c6938adc9daaa5,7632,4118,4017.0,3964.0,82688.83,3480.0,0.539570,0.526336,1.139080,23.761158
3,3f207df678b143eea3cee63160fa8bed,7617,4144,0.0,0.0,0.00,0.0,0.544046,0.000000,0.000000,0.000000
4,4d5c57ea9a6940dd891ad53e9dbe8da0,7593,7298,3331.0,3301.0,79283.59,2885.0,0.961148,0.438694,1.144194,27.481314
5,5a8bc65990b245e5a138643cd4eb9837,7618,6687,0.0,0.0,0.00,0.0,0.877789,0.000000,0.000000,0.000000
6,9b98b8c7a33c4b65b9aebfe6a799e6d9,7677,4171,4354.0,4303.0,77911.04,3784.0,0.543311,0.567149,1.137156,20.589598
7,ae264e3637204a6fb9bb56bc8210ddfd,7658,6716,3688.0,3635.0,88106.63,3177.0,0.876991,0.481588,1.144161,27.732650
8,f19421c1d4aa40978ebb69ca19b0e20d,7571,7264,4296.0,4262.0,83592.17,3741.0,0.959451,0.567428,1.139268,22.344873
9,fafdcd668e3743c1bb461111dcafc2a4,7597,7327,5317.0,5265.0,97892.24,4530.0,0.964460,0.699882,1.162252,21.609766


In [68]:
offer_funnel_subset = offer_funnel[['offer_id','viewed_rate','completion_rate','avg_transactions_per_customer','avg_spend_per_customer']]
offer_funnel_v2 = portfolio.merge(offer_funnel_subset, how='left', on='offer_id')
offer_funnel_v2

,reward,channels,difficulty,duration,offer_type,offer_id,viewed_rate,completion_rate,avg_transactions_per_customer,avg_spend_per_customer
0,10,"[email, mobile, social]",10,168,bogo,ae264e3637204a6fb9bb56bc8210ddfd,0.876991,0.481588,1.144161,27.732650
1,10,"[web, email, mobile, social]",10,120,bogo,4d5c57ea9a6940dd891ad53e9dbe8da0,0.961148,0.438694,1.144194,27.481314
2,0,"[web, email, mobile]",0,96,informational,3f207df678b143eea3cee63160fa8bed,0.544046,0.000000,0.000000,0.000000
3,5,"[web, email, mobile]",5,168,bogo,9b98b8c7a33c4b65b9aebfe6a799e6d9,0.543311,0.567149,1.137156,20.589598
4,5,"[web, email]",20,240,discount,0b1e1539f2cc45b7b9fa7c272da2e1d7,0.347287,0.446009,1.121558,29.515957
5,3,"[web, email, mobile, social]",7,168,discount,2298d6c36e964ae4a3e7e9706d1fb8c2,0.959587,0.674340,1.156299,20.482339
6,2,"[web, email, mobile, social]",10,240,discount,fafdcd668e3743c1bb461111dcafc2a4,0.964460,0.699882,1.162252,21.609766
7,0,"[email, mobile, social]",0,72,informational,5a8bc65990b245e5a138643cd4eb9837,0.877789,0.000000,0.000000,0.000000
8,5,"[web, email, mobile, social]",5,120,bogo,f19421c1d4aa40978ebb69ca19b0e20d,0.959451,0.567428,1.139268,22.344873
9,2,"[web, email, mobile]",10,168,discount,2906b810c7d4411798c6938adc9daaa5,0.539570,0.526336,1.139080,23.761158
